## Test the hybrid quasiparticle configuration system

Imports

In [121]:
from typing import List
import numpy as np
from scipy.sparse import lil_matrix
from NSMFermions.nuclear_physics_utils import SingleParticleState,get_twobody_nuclearshell_model
from NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian
from scipy.sparse.linalg import eigsh

In [122]:
class HybridQuasiParticleConverter():
    
    def __init__(self,):
        pass
    
    def initialize_shell(self,state_encoding:List,fermionic_subset:np.ndarray):
        
        self.fermionic_subset=fermionic_subset
        self.quasiparticle_subset= np.setdiff1d(np.arange(len(state_encoding)), fermionic_subset)
        print(self.quasiparticle_subset)
        #### nn and pp
        couples=[]

        for a,state_a in enumerate(np.array(state_encoding)[self.quasiparticle_subset]):
            for b,state_b in enumerate(np.array(state_encoding)[self.quasiparticle_subset]):
                a_corresponding_index=self.quasiparticle_subset[a]
                b_corresponding_index=self.quasiparticle_subset[b]
                if b_corresponding_index>a_corresponding_index:
                    _,_,ja,ma,_,tza=state_a
                    _,_,jb,mb,_,tzb=state_b
                    if ja==jb and ma==-mb and tza==tzb:
                        couples.append([a_corresponding_index,b_corresponding_index])     

        self.couples=couples



    def new_base_computation(self,base:np.ndarray):
        
        #indices=np.nonzero(base)[0]
        new_base=np.zeros(len(self.couples)+self.fermionic_subset.shape[0])
        value=np.sum(base[self.quasiparticle_subset])
        value2=np.sum(base[self.fermionic_subset])
    
                
        list_of_token_indices=[]
        
        for i in range(len(self.couples)):
            
            if base[self.couples[i][0]]+base[self.couples[i][1]]!=2 :
                continue
            else:
                new_base[i]+=1
                base[self.couples[i][0]]=0
                base[self.couples[i][1]]=0
        
        new_base[len(self.couples):]=base[self.fermionic_subset]
        
        if np.sum(new_base[:len(self.couples)])==value//2 and np.sum(new_base[len(self.couples):])==value2:
           return new_base
        
    def get_the_basis_matrix_transformation(self,basis:np.ndarray):
        
        self.quasiparticle_basis=[]
        self.rest_basis=[]
        
        for i,b in enumerate(basis):
            qp_base=self.new_base_computation(base=b.copy())
            
            if qp_base is not(None):

                self.quasiparticle_basis.append(qp_base)
            else:
                self.rest_basis.append(b.copy())
        self.quasiparticle_basis=np.asarray(self.quasiparticle_basis)
        
        self.rest_basis=np.asarray(self.rest_basis)
        
        self.particles2quasiparticles=lil_matrix((self.quasiparticle_basis.shape[0],basis.shape[0]))
        self.particles2restofstates=lil_matrix((self.rest_basis.shape[0],basis.shape[0]))
        qp_idx=0
        rest_idx=0
        for i,b in enumerate(basis):
            qp_base=self.new_base_computation(base=b.copy())
            
            if qp_base is not(None):
                self.particles2quasiparticles[qp_idx,i]=1.
                qp_idx+=1
            else:
                self.particles2restofstates[rest_idx,i]=1
                rest_idx+=1
                


Load the encoding

In [ ]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

HQPC=HybridQuasiParticleConverter()

index=np.array( [2,3,6,7,9,10])
index2=index+12

fermion_subset=np.concatenate((index,index2))
HQPC.initialize_shell(state_encoding=SPS.state_encoding,fermionic_subset=fermion_subset)
print(HQPC.couples)


[ 0  1  4  5  8 11 12 13 16 17 20 23]
[[np.int64(0), np.int64(5)], [np.int64(1), np.int64(4)], [np.int64(8), np.int64(11)], [np.int64(12), np.int64(17)], [np.int64(13), np.int64(16)], [np.int64(20), np.int64(23)]]


#### Test using the basis of the NSM Hamiltonian

In [156]:
size=len(SPS.energies)//2
nparticles_a=2
nparticles_b=2

In [127]:
twobody_matrix,_=get_twobody_nuclearshell_model(file_name=file_name)

Computing the matrix, pls wait... (u_u) 



100%|██████████| 24/24 [00:47<00:00,  1.96s/it]


In [166]:
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size,size_b=size,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
NSMHamiltonian.get_external_potential(SPS.energies)
NSMHamiltonian.get_twobody_interaction_optimized(twobody_matrix)
NSMHamiltonian.get_hamiltonian()

egs,psigs=NSMHamiltonian.get_spectrum(n_states=1)
print(egs)

HQPC.get_the_basis_matrix_transformation(NSMHamiltonian.basis)



Building two-body operator with 16496 terms...


100%|██████████| 16496/16496 [00:01<00:00, 11567.45it/s]

✅ Two-body operator built: shape=(640, 640), nnz=54112
[-41.39649007]


In [153]:
print(HQPC.quasiparticle_basis.shape)

(234, 18)


In [ ]:
hamiltonian_Q=HQPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ HQPC.particles2quasiparticles.T
egs,psi_Q=eigsh(hamiltonian_Q,k=1,which='SA')

print(egs)

psi_Q2particle=HQPC.particles2quasiparticles.T @ psi_Q

print(psi_Q2particle.shape)

print(1-np.linalg.norm(psigs[:,0].dot(psi_Q2particle[:,0]))**2)

[-38.60496514]
(640, 1)
0.7817558170399592
